In [12]:
import pandas as pd
import matplotlib.pyplot as plt


In [18]:
# read the csv file into a data frame
unfiltered_df = pd.read_csv('data\\lagos-rent.csv')
print(len(unfiltered_df))

53070


In [19]:
# get a sense/clearer picture of the data we are working with
print(unfiltered_df.columns)
print(unfiltered_df.head())

Index(['Title', 'More Info', 'Price', 'Serviced', 'Newly Built', 'Furnished',
       'Bedrooms', 'Bathrooms', 'Toilets', 'City', 'Neighborhood'],
      dtype='object')
                                               Title  \
0  Newly Built 4 Bedroom Semi Detached House With Bq   
1      Superb 4 Bedroom Semi Detached Duplex With Bq   
2    Furnished 2 Bedroom Terrace Duplex For Rent !!!   
3                       2 Bedroom Apartment For Rent   
4                  4 Bedroom Terrace Duplex For Rent   

                  More Info           Price  Serviced  Newly Built  Furnished  \
0  4 BEDROOM HOUSE FOR RENT  5,000,000/year         0            1          0   
1  4 BEDROOM HOUSE FOR RENT  5,000,000/year         0            1          0   
2  2 BEDROOM HOUSE FOR RENT       3,500,000         1            0          0   
3  2 BEDROOM HOUSE FOR RENT       2,700,000         1            1          0   
4  4 BEDROOM HOUSE FOR RENT       4,000,000         1            0          0   

  Bedroo

In [20]:
# let's perform some filtering on the data
# 1. limit the data to properties with 1 or more bedrooms
df = unfiltered_df[unfiltered_df['Bedrooms'].str.contains('\\d') & (unfiltered_df['Bedrooms'] != '0 beds')]
# 2. filter by the properties with yearly price
df = df[df['Price'].str.contains('\\d+/year', regex=True)]

In [21]:
# know the current size of our data after filtering
print(len(df))

27130


In [22]:
# Data formatting
# 1. format the Price column which is a string in the format 'price/year' into a float
df['Price'] = df['Price'].replace(',|/year', '', regex=True)
df['Price'] = df['Price'].astype(float)
# 2. format the Bedrooms column which is displayed as 'num beds' into an int
df['Bedrooms'] = df['Bedrooms'].str.extract('(\\d)')
df['Bedrooms'] = df['Bedrooms'].astype(int)

In [25]:
# preview data after filtering after filtering and formatting
print(df.head())

print(df['Price'])
print(df['Bedrooms'])

                                                Title  \
0   Newly Built 4 Bedroom Semi Detached House With Bq   
1       Superb 4 Bedroom Semi Detached Duplex With Bq   
6                                    5 Bedroom Duplex   
8                                 2 Bedroom Apartment   
11              Nicely Built 4 Bedroom Terrace Duplex   

                              More Info       Price  Serviced  Newly Built  \
0              4 BEDROOM HOUSE FOR RENT   5000000.0         0            1   
1              4 BEDROOM HOUSE FOR RENT   5000000.0         0            1   
6              5 BEDROOM HOUSE FOR RENT  15000000.0         0            0   
8   2 BEDROOM FLAT / APARTMENT FOR RENT   8000000.0         1            1   
11             4 BEDROOM HOUSE FOR RENT   4000000.0         1            0   

    Furnished  Bedrooms Bathrooms    Toilets   City   Neighborhood  
0           0         4   4 baths  5 Toilets  Lekki         Agungi  
1           0         4   4 baths  5 Toilets  Lekk

In [26]:
# get a description of the data pulling off unneeded columns first
description = df.drop(['Serviced', 'Newly Built', 'Furnished', 'More Info'], axis=1).describe()
print(description)

              Price      Bedrooms
count  2.713000e+04  27130.000000
mean   7.193710e+07      2.675599
std    1.032256e+10      1.270748
min    1.000000e+00      1.000000
25%    1.100000e+06      2.000000
50%    2.500000e+06      3.000000
75%    5.000000e+06      4.000000
max    1.700000e+12      9.000000


In [27]:
# get a list of unique cities as this will be used later on
cities = df['City'].unique().tolist()